# LPBF Optimizer: Complete Workflow Example

This notebook demonstrates the complete end-to-end workflow for LPBF process optimization using physics-informed neural networks and multi-objective optimization. The workflow includes:

1. Generating synthetic data (simulating FEA results)
2. Data preprocessing and exploration
3. Training a physics-informed neural network (PINN)
4. Using the PINN as a surrogate model for multi-objective optimization
5. Analyzing the Pareto-optimal solutions
6. Visualizing process-property relationships

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import torch
import yaml
from pathlib import Path
import time
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Configure paths
config_path = '../data/params.yaml'
data_dir = Path('../data')
processed_dir = data_dir / 'processed'
models_dir = data_dir / 'models'
results_dir = data_dir / 'optimized'

# Create directories if they don't exist
processed_dir.mkdir(parents=True, exist_ok=True)
models_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

# Load configuration
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    
# Set up matplotlib for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 1. Synthetic Data Generation

We'll generate synthetic data that mimics FEA simulation results for LPBF process outcomes. This simulates running hundreds of finite element analyses for different process parameter combinations.

In [ ]:
from generate_synthetic_data import SyntheticDataGenerator

# Initialize data generator
generator = SyntheticDataGenerator(config_path)

# For this demo, we'll use a smaller dataset
n_scan_vectors = 50
n_points_per_vector = 200

print(f"Generating dataset with {n_scan_vectors} scan vectors and {n_points_per_vector} points per vector...")
start_time = time.time()

# Generate the synthetic dataset
dataset_path = generator.generate(n_scan_vectors=n_scan_vectors, n_points_per_vector=n_points_per_vector)

elapsed_time = time.time() - start_time
print(f"Generation complete in {elapsed_time:.2f} seconds")
print(f"Dataset saved to: {dataset_path}")

## 2. Data Exploration

Now let's explore the dataset to understand its structure and the relationships between process parameters and outcomes.

In [ ]:
# Open the dataset
with h5py.File(dataset_path, 'r') as f:
    # Print dataset structure
    print("Dataset structure:")
    def print_structure(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"  {name}: shape={obj.shape}, dtype={obj.dtype}")
    f.visititems(print_structure)
    
    # Get metadata
    param_names = [name.decode('utf-8') for name in f['metadata/parameter_names'][:]]
    n_train = f['metadata'].attrs['n_train']
    n_val = f['metadata'].attrs['n_val']
    n_test = f['metadata'].attrs['n_test']
    
    # Load sample data for visualization
    # Use unique scan vectors to see parameter distributions
    train_scan_vectors = f['train/scan_vectors'][:]
    train_outputs = f['train/outputs'][:1000]  # Sample of the outputs
    
print(f"\nParameter names: {param_names}")
print(f"Dataset split: {n_train} training, {n_val} validation, {n_test} test samples")

### 2.1 Visualize Process Parameter Distributions

In [ ]:
# Plot distributions of process parameters
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, param in enumerate(param_names[:4]):  # Show first 4 parameters
    if i < len(axes):
        axes[i].hist(train_scan_vectors[:, i], bins=20, alpha=0.7)
        axes[i].set_title(f'Distribution of {param}')
        axes[i].set_xlabel(param)
        axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### 2.2 Visualize Process Outcome Distributions

In [ ]:
# Plot distributions of outputs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

outcome_names = ['Residual Stress (MPa)', 'Porosity (%)', 'Geometric Accuracy Ratio']

for i, name in enumerate(outcome_names):
    axes[i].hist(train_outputs[:, i], bins=30, alpha=0.7)
    axes[i].set_title(f'Distribution of {name}')
    axes[i].set_xlabel(name)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### 2.3 Explore Process-Property Relationships

Let's look at how key process parameters affect the outcomes.

In [ ]:
# Load more data for relationship exploration
with h5py.File(dataset_path, 'r') as f:
    # Get a subset of inputs and outputs
    inputs = f['train/inputs'][:2000]
    outputs = f['train/outputs'][:2000]
    coordinates = f['train/coordinates'][:2000]
    n_params = len(param_names)
    
    # Extract parameters portion of inputs
    params = inputs[:, :n_params]

# Create a figure with scatter plots showing relationships
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Laser power vs. Residual stress
axes[0, 0].scatter(params[:, 0], outputs[:, 0], alpha=0.5, c=params[:, 1], cmap='viridis')
axes[0, 0].set_title('Laser Power vs. Residual Stress')
axes[0, 0].set_xlabel('Laser Power (W)')
axes[0, 0].set_ylabel('Residual Stress (MPa)')
axes[0, 0].colorbar = plt.colorbar(axes[0, 0].collections[0], ax=axes[0, 0])
axes[0, 0].colorbar.set_label('Scan Speed (mm/s)')

# Scan speed vs. Porosity
axes[0, 1].scatter(params[:, 1], outputs[:, 1], alpha=0.5, c=params[:, 0], cmap='plasma')
axes[0, 1].set_title('Scan Speed vs. Porosity')
axes[0, 1].set_xlabel('Scan Speed (mm/s)')
axes[0, 1].set_ylabel('Porosity (%)')
axes[0, 1].colorbar = plt.colorbar(axes[0, 1].collections[0], ax=axes[0, 1])
axes[0, 1].colorbar.set_label('Laser Power (W)')

# Energy density vs. Geometric accuracy
energy_density = params[:, 0] / (params[:, 1] * params[:, 2]) # P/(v*h)
scatter = axes[0, 2].scatter(energy_density, outputs[:, 2], alpha=0.5, c=params[:, 3], cmap='coolwarm')
axes[0, 2].set_title('Energy Density vs. Geometric Accuracy')
axes[0, 2].set_xlabel('Energy Density (J/mm³)')
axes[0, 2].set_ylabel('Geometric Accuracy Ratio')
axes[0, 2].colorbar = plt.colorbar(scatter, ax=axes[0, 2])
axes[0, 2].colorbar.set_label('Scan Angle (°)')

# Hatch spacing vs. Residual stress
axes[1, 0].scatter(params[:, 2], outputs[:, 0], alpha=0.5, c=params[:, 0], cmap='viridis')
axes[1, 0].set_title('Hatch Spacing vs. Residual Stress')
axes[1, 0].set_xlabel('Hatch Spacing (mm)')
axes[1, 0].set_ylabel('Residual Stress (MPa)')
axes[1, 0].colorbar = plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])
axes[1, 0].colorbar.set_label('Laser Power (W)')

# Scan angle vs. Porosity
axes[1, 1].scatter(params[:, 3], outputs[:, 1], alpha=0.5, c=params[:, 1], cmap='plasma')
axes[1, 1].set_title('Scan Angle vs. Porosity')
axes[1, 1].set_xlabel('Scan Angle (°)')
axes[1, 1].set_ylabel('Porosity (%)')
axes[1, 1].colorbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
axes[1, 1].colorbar.set_label('Scan Speed (mm/s)')

# Cooling rate proxy (P/sqrt(v)) vs Geometric accuracy
cooling_rate_proxy = params[:, 0] / np.sqrt(params[:, 1]) # P/sqrt(v)
scatter = axes[1, 2].scatter(cooling_rate_proxy, outputs[:, 2], alpha=0.5, c=params[:, 2], cmap='coolwarm')
axes[1, 2].set_title('Cooling Rate Proxy vs. Geometric Accuracy')
axes[1, 2].set_xlabel('P/sqrt(v) (W·s^(1/2)/mm^(1/2))')
axes[1, 2].set_ylabel('Geometric Accuracy Ratio')
axes[1, 2].colorbar = plt.colorbar(scatter, ax=axes[1, 2])
axes[1, 2].colorbar.set_label('Hatch Spacing (mm)')

plt.tight_layout()
plt.show()

### 2.4 Visualize Spatial Variations

Now let's visualize how properties vary spatially across a sample.

In [ ]:
# Load spatial data for a single scan vector
with h5py.File(dataset_path, 'r') as f:
    # Get indices for the first scan vector
    scan_vector_0 = f['train/scan_vectors'][0]
    n_params = len(param_names)
    
    # Load all training data
    all_inputs = f['train/inputs'][:]
    all_outputs = f['train/outputs'][:]
    all_coords = f['train/coordinates'][:]
    
    # Find points that match our scan vector
    mask = np.all(np.isclose(all_inputs[:, :n_params], scan_vector_0), axis=1)
    matched_coords = all_coords[mask]
    matched_outputs = all_outputs[mask]
    
# Create contour plots to show spatial variations
# First, reshape to 3D grid
# Determine grid dimensions - assume cubic grid
points_per_dim = int(np.cbrt(len(matched_coords)))

# Extract x, y, z coordinates
x = matched_coords[:, 0].reshape(points_per_dim, points_per_dim, points_per_dim)
y = matched_coords[:, 1].reshape(points_per_dim, points_per_dim, points_per_dim)
z = matched_coords[:, 2].reshape(points_per_dim, points_per_dim, points_per_dim)

# Reshape outputs
stress = matched_outputs[:, 0].reshape(points_per_dim, points_per_dim, points_per_dim)
porosity = matched_outputs[:, 1].reshape(points_per_dim, points_per_dim, points_per_dim)
accuracy = matched_outputs[:, 2].reshape(points_per_dim, points_per_dim, points_per_dim)

# Create slices through the middle of the z-axis
z_mid_idx = points_per_dim // 2

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot residual stress
im0 = axes[0].contourf(x[:, :, z_mid_idx], y[:, :, z_mid_idx], stress[:, :, z_mid_idx], 50, cmap='hot')
axes[0].set_title('Residual Stress Distribution')
axes[0].set_xlabel('X (mm)')
axes[0].set_ylabel('Y (mm)')
plt.colorbar(im0, ax=axes[0], label='Stress (MPa)')

# Plot porosity
im1 = axes[1].contourf(x[:, :, z_mid_idx], y[:, :, z_mid_idx], porosity[:, :, z_mid_idx], 50, cmap='viridis')
axes[1].set_title('Porosity Distribution')
axes[1].set_xlabel('X (mm)')
axes[1].set_ylabel('Y (mm)')
plt.colorbar(im1, ax=axes[1], label='Porosity (%)')

# Plot geometric accuracy
im2 = axes[2].contourf(x[:, :, z_mid_idx], y[:, :, z_mid_idx], accuracy[:, :, z_mid_idx], 50, cmap='Blues')
axes[2].set_title('Geometric Accuracy Distribution')
axes[2].set_xlabel('X (mm)')
axes[2].set_ylabel('Y (mm)')
plt.colorbar(im2, ax=axes[2], label='Accuracy Ratio')

plt.tight_layout()
plt.show()

# Print the scan vector parameters for reference
scan_vector_info = {param: value for param, value in zip(param_names, scan_vector_0)}
print("Scan vector parameters for this spatial visualization:")
for param, value in scan_vector_info.items():
    print(f"  {param}: {value}")

## 3. PINN Model Training

Now we'll train a Physics-Informed Neural Network (PINN) to predict the process outcomes based on scan parameters and spatial coordinates.

In [ ]:
from pinn.model import PINN
from pinn.physics import compute_physics_loss
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load a subset of the data for demonstration
with h5py.File(dataset_path, 'r') as f:
    # Load inputs and outputs (limit to 5000 samples for faster training)
    train_inputs = torch.tensor(f['train/inputs'][:5000], dtype=torch.float32)
    train_outputs = torch.tensor(f['train/outputs'][:5000], dtype=torch.float32)
    val_inputs = torch.tensor(f['val/inputs'][:1000], dtype=torch.float32)
    val_outputs = torch.tensor(f['val/outputs'][:1000], dtype=torch.float32)
    
    # Extract scan vectors, coordinates, and time for physics loss
    n_params = len(param_names)
    train_scan_vectors = train_inputs[:, :n_params]
    train_coords = train_inputs[:, n_params:n_params+3]
    train_time = train_inputs[:, n_params+3:n_params+4]
    
    val_scan_vectors = val_inputs[:, :n_params]
    val_coords = val_inputs[:, n_params:n_params+3]
    val_time = val_inputs[:, n_params+3:n_params+4]

# Set device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model
model_config = config['model']
model = PINN(
    in_dim=model_config['input_dim'],
    out_dim=model_config['output_dim'],
    width=model_config['hidden_width'],
    depth=model_config['hidden_depth']
).to(device)

# Move data to device
train_inputs = train_inputs.to(device)
train_outputs = train_outputs.to(device)
train_scan_vectors = train_scan_vectors.to(device)
train_coords = train_coords.to(device)
train_time = train_time.to(device)

val_inputs = val_inputs.to(device)
val_outputs = val_outputs.to(device)
val_scan_vectors = val_scan_vectors.to(device)
val_coords = val_coords.to(device)
val_time = val_time.to(device)

# Create data loaders
train_dataset = TensorDataset(train_inputs, train_outputs, train_scan_vectors, train_coords, train_time)
val_dataset = TensorDataset(val_inputs, val_outputs, val_scan_vectors, val_coords, val_time)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define loss and optimizer
mse_loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

# Training parameters
n_epochs = 100  # Reduced for demonstration
physics_weight_schedule = np.linspace(0, 0.1, 30)  # Gradually increase physics weight
physics_weight_schedule = np.pad(physics_weight_schedule, (0, n_epochs - len(physics_weight_schedule)), 'constant', constant_values=0.1)

# Training loop
train_losses = []
val_losses = []
data_losses = []
physics_losses = []
best_val_loss = float('inf')

print("Starting training...")
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    epoch_data_loss = 0
    epoch_physics_loss = 0
    
    # Set physics weight for this epoch
    lambda_heat = physics_weight_schedule[epoch]
    lambda_stress = physics_weight_schedule[epoch]
    
    # Training loop
    for inputs, outputs, scan_vectors, coords, time in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(inputs)
        
        # Data loss
        data_loss = mse_loss(predictions, outputs)
        
        # Physics loss
        physics_loss = compute_physics_loss(
            model, 
            scan_vectors, 
            coords, 
            time, 
            config['material_properties'],
            lambda_heat=lambda_heat,
            lambda_stress=lambda_stress
        )
        
        # Total loss
        total_loss = data_loss + physics_loss
        
        # Backward and optimize
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip gradients
        optimizer.step()
        
        # Update epoch losses
        epoch_loss += total_loss.item()
        epoch_data_loss += data_loss.item()
        epoch_physics_loss += physics_loss.item()
    
    # Calculate average losses
    avg_loss = epoch_loss / len(train_loader)
    avg_data_loss = epoch_data_loss / len(train_loader)
    avg_physics_loss = epoch_physics_loss / len(train_loader)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, outputs, _, _, _ in val_loader:
            predictions = model(inputs)
            val_loss += mse_loss(predictions, outputs).item()
    
    avg_val_loss = val_loss / len(val_loader)
    
    # Update learning rate
    scheduler.step(avg_val_loss)
    
    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val_loss
        }, models_dir / 'demo_best_model.pt')
    
    # Record losses
    train_losses.append(avg_loss)
    val_losses.append(avg_val_loss)
    data_losses.append(avg_data_loss)
    physics_losses.append(avg_physics_loss)
    
    # Print progress
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}], "
              f"Loss: {avg_loss:.6f}, "
              f"Data Loss: {avg_data_loss:.6f}, "
              f"Physics Loss: {avg_physics_loss:.6f}, "
              f"Val Loss: {avg_val_loss:.6f}")

print(f"Training complete. Best validation loss: {best_val_loss:.6f}")

### 3.1 Visualize Training Progress

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 8))
plt.semilogy(train_losses, label='Training Loss')
plt.semilogy(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot data loss vs physics loss
plt.figure(figsize=(12, 8))
plt.semilogy(data_losses, label='Data Loss')
plt.semilogy(physics_losses, label='Physics Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Component (log scale)')
plt.title('Data Loss vs Physics Loss')
plt.legend()
plt.grid(True)
plt.show()

### 3.2 Evaluate Model Accuracy

Let's evaluate the model's prediction accuracy on the test dataset.

In [ ]:
# Load test data
with h5py.File(dataset_path, 'r') as f:
    test_inputs = torch.tensor(f['test/inputs'][:], dtype=torch.float32).to(device)
    test_outputs = torch.tensor(f['test/outputs'][:], dtype=torch.float32).to(device)

# Load best model
checkpoint = torch.load(models_dir / 'demo_best_model.pt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Make predictions
with torch.no_grad():
    predictions = model(test_inputs)

# Calculate metrics
predictions_np = predictions.cpu().numpy()
test_outputs_np = test_outputs.cpu().numpy()

# Calculate R² for each output
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2_values = []
rmse_values = []
mae_values = []
output_names = ['Residual Stress', 'Porosity', 'Geometric Accuracy']

for i, name in enumerate(output_names):
    r2 = r2_score(test_outputs_np[:, i], predictions_np[:, i])
    rmse = np.sqrt(mean_squared_error(test_outputs_np[:, i], predictions_np[:, i]))
    mae = mean_absolute_error(test_outputs_np[:, i], predictions_np[:, i])
    
    r2_values.append(r2)
    rmse_values.append(rmse)
    mae_values.append(mae)
    
    print(f"{name}: R² = {r2:.4f}, RMSE = {rmse:.4f}, MAE = {mae:.4f}")

# Visualize predictions vs. actual values
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (name, ax) in enumerate(zip(output_names, axes)):
    # Sample 1000 points for visualization
    idx = np.random.choice(len(predictions_np), size=1000, replace=False)
    ax.scatter(test_outputs_np[idx, i], predictions_np[idx, i], alpha=0.5)
    
    # Add perfect prediction line
    min_val = min(test_outputs_np[:, i].min(), predictions_np[:, i].min())
    max_val = max(test_outputs_np[:, i].max(), predictions_np[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    ax.set_title(f'{name} Prediction (R² = {r2_values[i]:.4f})')
    ax.set_xlabel('Actual Value')
    ax.set_ylabel('Predicted Value')
    ax.grid(True)

plt.tight_layout()
plt.show()

## 4. Process-Property Maps

Now that we have a trained model, we can use it to create detailed process-property maps to visualize how different parameter combinations affect outcomes.

In [ ]:
# Generate a grid of process parameters
# We'll focus on laser power and scan speed
param_bounds = config['optimizer']['param_bounds']

# Create grid for first two parameters (power and speed)
p_range = np.linspace(param_bounds['P'][0], param_bounds['P'][1], 50)
v_range = np.linspace(param_bounds['v'][0], param_bounds['v'][1], 50)
P, V = np.meshgrid(p_range, v_range)

# Get default values for other parameters
default_params = {}
for param in param_bounds:
    if param not in ['P', 'v']:
        default_params[param] = np.mean(param_bounds[param])

# Create test points
n_pts = P.shape[0] * P.shape[1]
test_points = np.zeros((n_pts, len(param_names)))
test_points[:, 0] = P.flatten()  # Power
test_points[:, 1] = V.flatten()  # Speed

# Fill in default values for other parameters
for i, param in enumerate(param_names):
    if i >= 2 and param in default_params:
        test_points[:, i] = default_params[param]

# Add dummy spatial coordinates (center point) and time
coords = np.zeros((n_pts, 3))
time = np.ones((n_pts, 1))

# Create input tensor
test_input = np.hstack([test_points, coords, time])
test_input_tensor = torch.tensor(test_input, dtype=torch.float32).to(device)

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(test_input_tensor)

# Reshape predictions for contour plots
pred_stress = predictions[:, 0].cpu().numpy().reshape(P.shape)
pred_porosity = predictions[:, 1].cpu().numpy().reshape(P.shape)
pred_accuracy = predictions[:, 2].cpu().numpy().reshape(P.shape)

# Create contour plots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Residual Stress
contour0 = axes[0].contourf(P, V, pred_stress, 50, cmap='hot')
axes[0].set_title('Residual Stress Map')
axes[0].set_xlabel('Laser Power (W)')
axes[0].set_ylabel('Scan Speed (mm/s)')
plt.colorbar(contour0, ax=axes[0], label='Stress (MPa)')

# Porosity
contour1 = axes[1].contourf(P, V, pred_porosity, 50, cmap='viridis')
axes[1].set_title('Porosity Map')
axes[1].set_xlabel('Laser Power (W)')
axes[1].set_ylabel('Scan Speed (mm/s)')
plt.colorbar(contour1, ax=axes[1], label='Porosity (%)')

# Geometric Accuracy
contour2 = axes[2].contourf(P, V, pred_accuracy, 50, cmap='Blues')
axes[2].set_title('Geometric Accuracy Map')
axes[2].set_xlabel('Laser Power (W)')
axes[2].set_ylabel('Scan Speed (mm/s)')
plt.colorbar(contour2, ax=axes[2], label='Accuracy Ratio')

plt.tight_layout()
plt.show()

## 5. Multi-objective Optimization

Finally, let's use our trained PINN model with the NSGA-III algorithm to find Pareto-optimal process parameters.

In [ ]:
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.optimize import minimize
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.visualization.scatter import Scatter
from pymoo.visualization.pcp import PCP

class SurrogateProblem(Problem):
    """Multi-objective optimization problem using PINN surrogate"""
    def __init__(self, model, param_bounds, param_names, device):
        # Extract parameter bounds
        self.n_var = len(param_bounds)
        self.param_names = param_names
        self.xl = np.array([param_bounds[p][0] for p in self.param_names])
        self.xu = np.array([param_bounds[p][1] for p in self.param_names])
        
        # Set objectives (all three outcomes)
        self.n_obj = 3
        
        # No constraints
        self.n_constr = 0
        
        # Store the surrogate model
        self.model = model
        self.device = device
        
        super().__init__(
            n_var=self.n_var,
            n_obj=self.n_obj,
            n_constr=self.n_constr,
            xl=self.xl,
            xu=self.xu
        )
    
    def _evaluate(self, x, out, *args, **kwargs):
        # Convert to tensor
        x_tensor = torch.tensor(x, dtype=torch.float32, device=self.device)
        
        # Add dummy spatial coordinates and time
        batch_size = x_tensor.shape[0]
        coords = torch.zeros(batch_size, 3, device=self.device)  # Origin point
        time = torch.ones(batch_size, 1, device=self.device)     # Final time step
        
        # Forward pass through the model
        with torch.no_grad():
            model_input = torch.cat([x_tensor, coords, time], dim=1)
            predictions = self.model(model_input)
        
        # Extract objective values
        objectives = predictions.cpu().numpy()
        
        # Negate geometric accuracy (we want to maximize it)
        objectives[:, 2] = -objectives[:, 2]  # Negate for maximization
        
        # Set the objective values
        out["F"] = objectives

# Set up problem
problem = SurrogateProblem(
    model=model,
    param_bounds={name: param_bounds[name] for name in param_names},
    param_names=param_names,
    device=device
)

# Get reference directions for 3 objectives
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=8)

# Create the algorithm
algorithm = NSGA3(
    pop_size=100,
    ref_dirs=ref_dirs,
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=15),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

# Run the optimization (reduced generations for demo)
n_gen = 10
print(f"Running optimization for {n_gen} generations...")
start_time = time.time()

res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=('n_gen', n_gen),
    seed=42,
    save_history=True,
    verbose=True
)

elapsed_time = time.time() - start_time
print(f"Optimization complete in {elapsed_time:.2f} seconds")
print(f"Number of solutions on Pareto front: {len(res.X)}")

### 5.1 Visualize Pareto Front

In [ ]:
# Extract results
X = res.X  # Parameter values
F = res.F  # Objective values

# Negate back the geometric accuracy for visualization
F[:, 2] = -F[:, 2]

# Create Pareto front visualization
fig = plt.figure(figsize=(14, 12))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    F[:, 0],  # Residual stress
    F[:, 1],  # Porosity
    F[:, 2],  # Geometric accuracy
    c=X[:, 0],  # Color by laser power
    cmap='viridis',
    s=50
)

ax.set_xlabel('Residual Stress (MPa)')
ax.set_ylabel('Porosity (%)')
ax.set_zlabel('Geometric Accuracy')
ax.set_title('Pareto Front of Optimal Solutions')

# Add colorbar
cbar = fig.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('Laser Power (W)')

plt.tight_layout()
plt.show()

### 5.2 Parallel Coordinates Plot

A Parallel Coordinates Plot helps visualize the relationships between process parameters and outcomes for multi-dimensional data.

In [ ]:
# Create a parallel coordinates plot
fig, ax = plt.subplots(figsize=(16, 8))

# Combine parameters and objectives
param_obj_data = np.hstack([X, F])

# Column names
columns = list(param_names) + ['Stress', 'Porosity', 'Accuracy']

# Normalize data for visualization
param_obj_norm = np.zeros_like(param_obj_data)
for i in range(param_obj_data.shape[1]):
    min_val = np.min(param_obj_data[:, i])
    max_val = np.max(param_obj_data[:, i])
    if max_val > min_val:
        param_obj_norm[:, i] = (param_obj_data[:, i] - min_val) / (max_val - min_val)
    else:
        param_obj_norm[:, i] = 0.5

# Create parallel coordinates plot
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm

# Value for coloring (e.g., energy density)
energy_density = X[:, 0] / (X[:, 1] * X[:, 2])  # P/(v*h)
scaler = MinMaxScaler()
colors = scaler.fit_transform(energy_density.reshape(-1, 1)).flatten()

# Plot parallel coordinates
x = np.arange(param_obj_norm.shape[1])
for i, (y, c) in enumerate(zip(param_obj_norm, colors)):
    ax.plot(x, y, color=cm.viridis(c), alpha=0.7)

# Set x-ticks and labels
ax.set_xticks(x)
ax.set_xticklabels(columns, rotation=45)

# Set y-ticks and labels for parameters
y_ticks = np.linspace(0, 1, 5)
y_labels = []
for i, col in enumerate(columns):
    if i < len(param_names):  # Parameters
        bounds = param_bounds[param_names[i]]
        labels = [f"{bounds[0]:.1f}", "", f"{np.mean(bounds):.1f}", "", f"{bounds[1]:.1f}"]
    else:  # Objectives
        min_val = np.min(param_obj_data[:, i])
        max_val = np.max(param_obj_data[:, i])
        labels = [f"{min_val:.2f}", "", f"{(min_val+max_val)/2:.2f}", "", f"{max_val:.2f}"]
    
    y_labels.append(labels)

# Set titles and labels
ax.set_title('Parallel Coordinates Plot of Pareto-Optimal Solutions', fontsize=14)
ax.grid(True)

# Add a colorbar
sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(energy_density.min(), energy_density.max()))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Energy Density (J/mm³)')

plt.tight_layout()
plt.show()

### 5.3 Display Optimal Parameter Sets

In [ ]:
# Create a DataFrame to display the Pareto-optimal solutions
param_cols = {name: X[:, i] for i, name in enumerate(param_names)}
obj_cols = {
    'Residual_Stress': F[:, 0],
    'Porosity': F[:, 1],
    'Geometric_Accuracy': F[:, 2]
}

# Calculate energy density
energy_density = X[:, 0] / (X[:, 1] * X[:, 2])  # P/(v*h)
obj_cols['Energy_Density'] = energy_density

# Create DataFrame
results_df = pd.DataFrame({**param_cols, **obj_cols})

# Display the top 10 solutions by different criteria
print("Top 10 solutions with LOWEST RESIDUAL STRESS:")
display(results_df.sort_values('Residual_Stress').head(10))

print("\nTop 10 solutions with LOWEST POROSITY:")
display(results_df.sort_values('Porosity').head(10))

print("\nTop 10 solutions with HIGHEST GEOMETRIC ACCURACY:")
display(results_df.sort_values('Geometric_Accuracy', ascending=False).head(10))

# Save results to CSV
results_path = results_dir / 'pareto_optimal_solutions.csv'
results_df.to_csv(results_path, index=False)
print(f"\nResults saved to {results_path}")

## 6. Selecting Optimal Process Parameters for Different Use Cases

Based on our multi-objective optimization, we can identify optimal parameter sets for different application requirements.

In [ ]:
# Define a few "use cases" with different priorities
use_cases = {
    'Structural': {'weights': [0.6, 0.2, 0.2]},  # Prioritize low stress
    'Fluid_Handling': {'weights': [0.2, 0.6, 0.2]},  # Prioritize low porosity
    'Precision_Parts': {'weights': [0.2, 0.2, 0.6]}  # Prioritize geometric accuracy
}

# Normalize the objectives for weighted scoring
stress_norm = (F[:, 0] - F[:, 0].min()) / (F[:, 0].max() - F[:, 0].min())
porosity_norm = (F[:, 1] - F[:, 1].min()) / (F[:, 1].max() - F[:, 1].min())
# For accuracy, we want to maximize (not minimize), so invert the normalization
accuracy_norm = 1 - (F[:, 2] - F[:, 2].min()) / (F[:, 2].max() - F[:, 2].min())

norm_objectives = np.column_stack([stress_norm, porosity_norm, accuracy_norm])

# Calculate weighted scores for each use case
for case, details in use_cases.items():
    weights = np.array(details['weights'])
    scores = np.sum(norm_objectives * weights, axis=1)
    best_idx = np.argmin(scores)  # Lower score is better
    
    print(f"\nOptimal parameters for {case} use case:")
    print(f"  Scores weighted by {weights}")
    for i, param in enumerate(param_names):
        print(f"  {param}: {X[best_idx, i]:.4f}")
    print("\nExpected outcomes:")
    print(f"  Residual Stress: {F[best_idx, 0]:.2f} MPa")
    print(f"  Porosity: {F[best_idx, 1]:.4f} %")
    print(f"  Geometric Accuracy: {F[best_idx, 2]:.4f}")
    print(f"  Energy Density: {energy_density[best_idx]:.4f} J/mm³")

## 7. Summary and Conclusion

In this notebook, we've demonstrated the complete workflow for LPBF process optimization using physics-informed neural networks:

1. We generated synthetic data that mimics FEA simulation results for LPBF process outcomes.
2. We explored the data to understand relationships between process parameters and outcomes.
3. We trained a physics-informed neural network that incorporates both data-driven learning and physical constraints.
4. We used the trained PINN as a surrogate model for multi-objective optimization using NSGA-III.
5. We visualized the Pareto-optimal solutions and identified optimal parameter sets for different use cases.

This approach offers several key advantages:

- **Efficiency**: Using a PINN surrogate avoids running thousands of computationally expensive FEA simulations.
- **Physics-based**: The model respects physical laws even in regions without data points.
- **Multi-objective**: The optimization balances competing objectives like minimizing stress and porosity while maximizing geometric accuracy.
- **Interpretable**: The process-property maps provide insights into how different parameters affect outcomes.

The optimal process parameters identified can be used to guide real LPBF manufacturing processes, potentially saving time and resources while improving part quality.